<a href="https://colab.research.google.com/github/ChuLinh02/GraphMining/blob/main/HW05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CSC17103 - Colab 5 - Phân hoạch và phát hiện cộng đồng trên đồ thị (Tiếp theo)**

Đây là Bài tập 05 trong môn học Khai thác Dữ liệu Đồ thị (Graph Mining) - CSC1703 tại Khoa Công nghệ Thông tin, Trường Đại học Khoa học Tự nhiên, Đại học Quốc gia TP.HCM

(Cập nhật lần cuối: 21/07/2024)

Họ tên:

MSSV:

## **Cách thực hiện và nộp bài tập**

&#9889; Chú ý, bạn nên thực hiện theo sự hướng dẫn dưới đây. Nếu có bất kỳ điều gì không rõ, bạn cần liên hệ ngay trợ giảng/ giáo viên thực hành để được hỗ trợ kịp thời.

**Cách làm bài**

Bạn sẽ làm trực tiếp trên file notebook này. Đầu tiên, bạn điền họ tên và MSSV vào phần đầu file ở bên trên. Trong file, bạn làm bài ở những chỗ có ghi là:
```python
# YOUR CODE HERE
raise NotImplementedError()
```
hoặc đối với những phần code không bắt buộc thì là:
```python
# YOUR CODE HERE (OPTION)
```
hoặc đối với markdown cell thì là:
```markdown
YOUR ANSWER HERE
```

**Cách nộp bài**

Trước khi nộp bài, bạn chọn `Kernel` - `Restart Kernel & Run All Cells` đối với trường hợp sử dụng môi trường local hoặc `Runtime` - `Restart session and run all` đối với môi trường Google Colab, để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, bạn tạo thư mục nộp bài theo cấu trúc sau:
- Thư mục `MSSV` (ví dụ, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`)
    - File `HW05.ipynb` (không cần nộp các file khác)

Cuối cùng, bạn nén thư mục `MSSV` này lại với định dạng nén là .zip (chứ không được là .rar hay các định dạng khác) và nộp ở link trên moodle. \
<font color=red>Bạn lưu ý tuân thủ chính xác qui định nộp bài này.</font>

## Setup và Import các thứ

In [ ]:
# Upgrade packages
# !pip install --upgrade scipy networkx

In [ ]:
# Import
import numpy as np # For numerical computation
import networkx as nx # For network/ graph manipulation
import matplotlib.pyplot as plt # For visualization
import scipy.stats as st
from scipy.stats import ks_2samp
from zlib import adler32

from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform

import pandas as pd
import seaborn as sns

from IPython.display import clear_output

from collections import Counter
import random

plt.style.use("fast")

from typing import List, Tuple, Dict

%matplotlib inline

In [ ]:
# Make sure something
assert nx.__version__ > '2.5'

Trước khi bắt đầu, bạn copy lại hàm `eigenLaplacian` đã cài đặt trước đó nhé!

In [ ]:
def eigenLaplacian(G: nx.Graph) -> Tuple[np.ndarray, np.ndarray]:
    # YOUR CODE HERE
    raise NotImplementedError()

## Task 6. Gom cụm phổ (Spectral clustering)

Gom cụm phổ hay Spectral clustering là một thuật toán mạnh mẽ phục vụ cho tác vụ phát hiện cộng đồng mà dựa trên việc cực tiểu lát cắt đồ thị. Nó dựa trên vector riêng Laplacian, một cách nói cụ thể hơn, dấu của vector riêng nhỏ thứ hai quyết định phân hoạch đồ thị thành hai cụm.

Trong phần này, ta xem xét một dataset nhân tạo dạng đường tròn như sau:

In [ ]:
from sklearn import datasets

circles, truth_labels = datasets.make_circles(
    n_samples=300, factor=0.6, noise=0.05, random_state=0
)

plt.figure(figsize=(6, 6))
plt.scatter(circles[:, 0], circles[:, 1], c=truth_labels,
            cmap='viridis', edgecolor='k')
plt.title('Scatter Plot of Generated Circles Dataset')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

Chúng ta hãy cô lập các điểm dữ liệu trong vòng tròn trung tâm bằng cách sử dụng gom cụm phổ. Bất kỳ tập dữ liệu dựa trên khoảng cách nào cũng có thể được biểu diễn dưới dạng đồ thị có các cạnh giữa các lân cận.

Bạn cần hoàn thành hàm `customGraph` mà nhận đầu vào là một mảng np.array với các điểm tọa độ x-y và trả về một đồ thị liên thông.

_Gợi ý: bạn thử áp dụng hàm `sklearn.neighbors.kneighbors_graph`_

_Nhận xét: bạn có thể đề xuất sử dụng khoảng cách của riêng bạn!_

In [ ]:
from sklearn.neighbors import kneighbors_graph

def customGraph(data: np.ndarray) -> nx.Graph:
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test
circles, truth_labels = datasets.make_circles(n_samples=300, factor=0.6,
                                              noise=0.05, random_state=0)
circlesG = customGraph(circles)
assert circlesG.number_of_nodes() == 300, "The graph should have 300 nodes"
assert nx.is_connected(circlesG), "The graph should be connected"

Trực quan hóa giá trị trong vector riêng nhỏ thứ hai của Laplacian.

In [ ]:
vecs, vals = eigenLaplacian(circlesG)

plt.figure(figsize=(8, 5))
plt.scatter(np.arange(300), vecs[:, 1], s=10, color='tab:blue', edgecolor='k')
plt.title('Second Smallest Eigenvector')
plt.xlabel('Data Point')
plt.ylabel('Value')
plt.grid(True)
plt.show()

Kế tiếp, bạn viết một hàm `spectralTwoClusters` mà nhận đầu vào là một mảng np.array vector riêng có thứ tự và trả về mảng np.array nhãn của các đỉnh.

In [ ]:
def spectralTwoClusters(vecs: np.ndarray) -> np.ndarray:
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test
circles, truth_labels = datasets.make_circles(n_samples=300, factor=0.6,
                                              noise=0.05, random_state=0)
circlesG = customGraph(circles)
vecs, vals = eigenLaplacian(circlesG)
labels = spectralTwoClusters(vecs)

assert abs(np.corrcoef(truth_labels, labels)[0, 1]) > 0.95

plt.figure(figsize=(6, 6))
plt.scatter(circles[:, 0], circles[:, 1], c=labels)
plt.title('Correlation: {:.2f}'.format(abs(np.corrcoef(truth_labels, labels)[0, 1])))
plt.show()

Bạn có nhận xét gì?

YOUR ANSWER HERE

_Ghi chú: ở đây có hai hướng trong xây dựng đồ thị: 1) lựa chọn các cặp đỉnh để liên kết lại với nhau, và 2) chọn trọng số trong việc khởi tạo cạnh. Hãy thử gán tham số `factor=0.7` trong `datasets.make_circles` và kiểm tra lại một lần nữa để đạt được tương quan 0.9. Kết hợp `sklearn.neighbors.kneighbors_graph` với `scipy.spatial.distance_matrix` hay `sklearn.metrics.pairwise.rbf_kernel`. Nếu bạn quan tâm, hãy đọc phân tích chi tiết [ở đây]((http://www2.imm.dtu.dk/projects/manifold/Papers/Laplacian.pdf))_

Sau khi thực hiện xong phần ghi chú, bạn để lại nhận xét của bạn ở phía dưới (phần này không tính điểm, chỉ mang tính chất điểm cộng).

YOUR ANSWER HERE

## Task 7. Các ánh xạ trị riêng Laplacian (Laplacian Eigenmaps)

Trong task trước, ta đã thấy cách để mà phân chia một đồ thị thành hai phần. Bây giờ, chúng ta xem xét một cách để có nhiều phân hoạch hơn trong một đồ thị như sau:

In [ ]:
quadG = nx.disjoint_union_all([nx.complete_graph(5),
                               nx.complete_graph(5),
                               nx.complete_graph(5),
                               nx.complete_graph(5)])

quadG.add_edge(0, 5)
quadG.add_edge(6, 10)
quadG.add_edge(11, 16)
quadG.add_edge(17, 1)

plt.figure(figsize=(5, 5))

nx.draw_kamada_kawai(
    quadG,
    node_color='white',
    edgecolors='black',
    node_size=100
)

plt.title('Quad Complete Graphs Connected')

plt.show()

Việc phân hoạch hơn nữa có thể được thực hiện bằng nhiều hướng tiếp cận khác nhau. Ví dụ như, ta có thể lặp lại thủ tục gom cụm trên các hai đồ thị con phân biệt và chọn một lát cắt đồ thị nhỏ nhất và sau đó tiếp tục lặp lại. Một cách khác là lấy ánh xạ trị riêng Laplacian và áp dụng K-means. Ta sẽ xem cách này hoạt động như thế nào:

1. Tính toán Laplacian chuẩn hóa dựa trên công thức
$$L = D^{-1/2}(D - A)D^{-1/2}$$
2. Xếp chồng các vector riêng  $L$ thành ma trận $(x_1, x_2, \dots)$ theo chiều giảm dần giá trị của các trị riêng.
3. Nhân dòng $i$-th bởi $\frac{1}{\sqrt{d_i}}$ trong đó $d_i$ là bậc của đỉnh $i$
* $k$ của ma trận tính từ dòng thứ hai này trở đi được gọi là vector _Laplacian Eigenmaps (hay còn gọi là Spectral Embedding)_
4. Ném các eigenmaps này vào Kmeans.
5. Các nhãn đỉnh được khớp bởi Kmeans.

Bây giờ, bạn viết một hàm `normLaplacian` mà nhận đầu vào là một ma trận kề $A$ và trả về một tuple với hai mảng numpy. Mảng đầu tiên là ma trận Laplacian đã được chuẩn hóa, mảng còn lại là một chuỗi bậc đỉnh.


In [ ]:
def normLaplacian(A: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test
A = nx.to_numpy_array(quadG)
L, degree_seq = normLaplacian(A)

assert degree_seq.shape == (20,), "Degree sequence should have 20 elements"
assert L.shape == (20, 20), "Laplacian matrix should be 20x20"
assert np.diagonal(L).sum() == 20, "Sum of the diagonal elements should be 20"
assert round(L[:, 2].sum(), 4) == 0.0528, (
    "Sum of the elements in the 3rd column should be 0.0528"
)

Tiếp theo, bạn hoàn thành hàm `spectralEmbedding` mà nhận đầu vào  mảng np.array `L` là Laplacian đã được chuẩn hóa, `degree_seq` là chuỗi bậc đỉnh, và `n_components` là số lượng vector đầu ra. Hàm này trả về một mảng np.array các Eigenmaps Laplacian

In [ ]:
def spectralEmbedding(L: np.ndarray, degree_seq: np.ndarray, n_components: int) -> np.ndarray:
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test
embedding = spectralEmbedding(L, degree_seq, 3)
assert embedding.shape == (20, 3), "Embedding matrix should be 20x3"

assert embedding[:, 0].max(
) > 0, "Max value in the first dimension should be > 0"
assert embedding[:, 0].min(
) < 0, "Min value in the first dimension should be < 0"
assert embedding[:, 1].max(
) > 0, "Max value in the second dimension should be > 0"
assert embedding[:, 1].min(
) < 0, "Min value in the second dimension should be < 0"

assert (embedding[:, 0] > 0).sum(
) == 10, "Number of positive values in the first dimension should be 10"
assert (embedding[:, 1] > 0).sum(
) == 10, "Number of positive values in the second dimension should be 10"
assert (embedding[:, 2] > 0).sum(
) == 10, "Number of positive values in the third dimension should be 10"

Giờ chúng ta ngắm nhìn vào một cặp biểu đồ của spectral embedding. Ta định nghĩa hàm `spectralClustering` để thực hiện gom cụm.

In [ ]:
def spectralClustering(G: nx.Graph, n_clusters: int, n_components: int) -> np.ndarray:
    """Thực hiện gom cụm phổ với một đồ thị đầu vào.

    Parameters
    ----------
    G
        (nx.Graph) Đồ thị đầu vào.
    n_clusters
        (int) Số lượng clusters.
    n_components
        (int) Số lượng thành phần trong spectral embedding.

    Returns
    -------
        (np.ndarray) Một mảng nhãn cụm cho mỗi nút.
    """
    A = nx.to_numpy_array(G)
    L, degree_seq = normLaplacian(A)
    embedding = spectralEmbedding(L, degree_seq, n_components)
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(embedding)
    return kmeans.labels_

In [ ]:
embedding = spectralEmbedding(L, degree_seq, 3)
labels = spectralClustering(quadG, 4, 3)
assert np.all(labels[:5] == labels[0]), "First complete graph nodes have different labels"
assert np.all(labels[5:10] == labels[5]), "Second complete graph nodes have different labels"
assert np.all(labels[10:15] == labels[10]), "Third complete graph nodes have different labels"
assert np.all(labels[15:20] == labels[15]), "Fourth complete graph nodes have different labels"

In [ ]:
embedding = spectralEmbedding(L, degree_seq, 3)
labels = spectralClustering(quadG, 4, 3)

pair_data = pd.DataFrame(
    np.hstack([embedding, labels[:, None]]),
    columns=['x2', 'x3', 'x4', 'label']
)

pair_data['label'] = pair_data['label'].astype('category')

plt.figure(figsize=(10, 10))

pairplot = sns.pairplot(
    pair_data,
    hue='label',
    diag_kind=None
)

# pairplot._legend.remove()

plt.show()

Chúng ta có thể thấy bốn cụm riêng biệt trong các ô, đó là lý do tại sao chúng ta lại sử dụng KMeans, nhưng các thuật toán phân cụm khác cũng có thể được áp dụng ở đây.

In [ ]:
plt.figure(figsize=(10, 10))
nx.draw_kamada_kawai(
    quadG,
    cmap=plt.cm.rainbow,
    node_color=labels,
    edgecolors='black',
    node_size=100)

Và bây giờ, ta xem thử spectral clustering hoạt động như thế nào trên đồ thị Les Miserables.

In [ ]:
G = nx.Graph()
G.add_edges_from(nx.les_miserables_graph().edges)
pos = nx.kamada_kawai_layout(G)

plt.figure(figsize=(8, 8))

node_colors = spectralClustering(G, 6, 5)

nodes = nx.draw_networkx_nodes(
    G,
    pos,
    cmap=plt.cm.rainbow,
    node_color=node_colors,
    node_size=100,
    linewidths=1,
    edgecolors='black'
)

nx.draw_networkx_edges(
    G,
    pos,
    alpha=0.2,
    edge_color='black'
)

plt.axis('off')

plt.legend(*nodes.legend_elements(), title="Clusters")

plt.show()

Bạn có nhận xét gì?

YOUR ANSWER HERE

## Task 8. Gọm cụm kết tụ (Agglomerative clustering)

Gom cụm kết tục là một thủ tục lặp đi lặp lại, liên kết các cụm tương tự bằng cách sử dụng ma trận tương đồng. Trong bài tập này, bởi vì chúng ta sử dụng `sklearn.cluster.AgglomerativeClustering` nên chúng ta cần truyền vào một ma trận _khoảng cách_ Do đó, trước tiên ta cần chọn một độ đo tương đồng và sau đó chuyển đổi nó về một khoảng cách. Bây giờ, chúng ta sẽ thực hiện nó thông qua độ đo SimRank mà được định nghĩa như sau: _hai đối tượng được xem là giống nhau (gần nhau) nếu chúng được tham chiếu bởi những đối tượng giống nhau._ Bởi vì SimRank có giá trị nằm trong khoảng [0, 1], nên ta sẽ định nghĩa một khoảng cách như sau:
$$\text{Distance}= 1 - \text{SimRank}$$

Bây giờ bạn hoàn thành một hàm `simrankDistance` mà nhận đầu vào là một đồ thị và trả về một ma trận khoảng cách dựa trên SimRank. Sử dụng `nx.simrank_similarity` để tính giá trị SimRank và sau đó chuyển đổi về mảng numpy. Ma trận khoảng cách nên đối xứng _một cách tuyệt đối_.

In [ ]:
def simrankDistance(G: nx.Graph) -> np.ndarray:
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test
distance = simrankDistance(G)
assert distance.shape == (77, 77)
assert np.diagonal(distance).sum() == 0
assert np.allclose(distance, distance.T, rtol=0, atol=0)
assert distance[52, 68] <= distance[1, 19]
assert distance[45, 29] <= distance[38, 46]

Sau đó, ta thực hiện gom cụm các đỉnh bằng cách sử dụng `scipy.cluster.hierarchy.linkage`. Method này nhận đầu vào là các khoảng cách và trả về một mảng numpy array là ma trận `Z` trong đó mọi dòng thứ $i$ bao gồm:
* `Z[i, 0]` và `Z[i, 1]` là chỉ số của các cụm được kết hợp.
* `Z[i, 2]` là khoảng cách giữ các cụm được kết hợp.
* `Z[i, 3]` là số lượng quan trắc trong các cụm được kết hợp.


Cụm có chỉ số `n+i` được mô tả ở hàng thứ $i$ của ma trận `Z`. Các cụm có chỉ số nhỏ hơn `n+1` là các đỉnh khởi tạo. Phương thức `scipy.cluster.hierarchy.dendrogram` sử dụng ma trận `Z` để vẽ sơ đồ dendrogram. Giờ ta sẽ xem xét dendrogram của đồ thị Les Miserables.

In [ ]:
plt.figure(figsize=(16, 6))
linked = linkage(squareform(distance), method='complete')
dendrogram(
    linked,
    labels=list(G.nodes),
    leaf_font_size=12
)
plt.axhline(y=0.89, color='tab:red', linestyle='--', linewidth=1)
plt.xlabel('Node')
plt.ylabel('Distance')
plt.title('Dendrogram')
plt.show()

Dựa trên hình trực quan, chúng ta cần chọn một đường ngang mà cho ta sự phân hoạch của đồ thị. Có hai cách tiếp cận: chọn số lượng cụm và chọn khoảng cách tối đa giữa các cụm.

Ví dụ: nếu chúng ta chọn một đường ngang có khoảng cách tối đa 0,89 (đường màu đỏ trên biểu đồ) thì số cụm sẽ là 4.

Bây giờ, bạn hoàn thành hàm `agglomerativeClustering` mà nhận vào một ma trận khoảng cách, cực đại khoảng cách giữa các cụm và trả về một mảng numpy array chứa các nhãn của các cụm. Bạn lưu ý sử dụng  `sklearn.cluster.AgglomerativeClustering` với complete linkage.

In [ ]:
def agglomerativeClustering(distance: np.ndarray, max_distance: float) -> np.ndarray:
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test
assert len(set(agglomerativeClustering(distance, 0.8))) == 17
assert len(set(agglomerativeClustering(distance, 0.89))) == 4

Xem xét đồ thị với khoảng cách cực đại 0.87.

In [ ]:
labels = agglomerativeClustering(distance, 0.87)
plt.figure(figsize=(8, 8))
nx.draw_networkx_nodes(
    G,
    pos,
    cmap=plt.cm.rainbow,
    node_color=labels,
    node_size=100,
    linewidths=1,
    edgecolors='black'
)
nx.draw_networkx_edges(
    G,
    pos,
    alpha=0.2,
    edge_color='black'
)
plt.axis('off')
plt.legend(*nodes.legend_elements(), title="Clusters", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

Bạn có nhận xét gì không?

YOUR ANSWER HERE

## Task 9. Phương pháp Louvain

[Phương pháp Louvain](https://en.wikipedia.org/wiki/Louvain_method) là một trong những công trình nổi tiếng trong nghiên cứu phát hiện cộng đồng. Nó là một phương pháp đơn giản để rút trích cấu trúc cộng đồng của các mạng lưới lớn. Phương pháp này là một phương pháp thuộc nhóm sử dụng heuristic dựa trên tối ưu mô-đun hóa (modularity optimization).

Trong phần này chúng ta sẽ xem xét phương pháp này trên một đồ thị phân dạng mà có thể biểu diễn một số cấu trúc sinh học.

In [ ]:
def fractalGraph(n: int, k: int, G: nx.Graph) -> nx.Graph:
    if k == 0:
        return G
    for node in list(G.nodes):
        newG = nx.complete_graph(n)
        newG = nx.relabel_nodes(
            newG, {node: max(G.nodes)+node+1 for node in newG.nodes})
        G = nx.union(G, newG)
        for edge in G.edges(node):
            G.add_edge(list(G.nodes)[-1], edge[1])
        G.remove_node(node)
    return fractalGraph(n, k-1, G)

In [ ]:
n = 4
fractalG = fractalGraph(n, 3, nx.complete_graph(n))

plt.figure(figsize=(8, 8))
frac_pos = nx.kamada_kawai_layout(fractalG)
nx.draw_networkx_nodes(fractalG, frac_pos, node_color='white',
                       edgecolors='black', node_size=100)
nx.draw_networkx_edges(fractalG, frac_pos, alpha=0.3)
plt.axis('off')
plt.show()

Phương pháp Louvain là một thuật toán tham lam dựa trên độ lợi modularity. Ta có thể mô tả nó thành từng pha như sau:
- Pha 1:
    - Bước 1: Đặt mỗi đỉnh trong cộng đồng của chính nó.
    - Bước 2: Với mỗi đỉnh, tính toán độ lợi modularity bằng cách loại bỏ nó từ cộng đồng của nó và thay thế nó trong cộng đồng của láng giềng.
    - Bước 3: Đặt đỉnh vào trong cộng đồng với độ lợi modularity cao nhất.
    - Bước 4: Lặp lại bước 2, 3 cho đến khi độ lợi modularity không tăng được nữa.

- Pha 2:
    - Bước 1: Gộp các cộng đồng thành "siêu đỉnh"
    - Bước 2: Chuyển các cạnh bên trong cộng đồng thành khuyên, cạnh giữa các cộng đồng thành các cạnh song song.

Lặp lại Pha 1, 2 cho đến khi độ lợi modularity không tăng được nữa.

In [ ]:
def louvainMethod(G: nx.Graph) -> Tuple[np.ndarray, np.ndarray, nx.Graph]:

    # Pha 1
    communities = unfoldedCommunities(G)
    labels = []
    for node in G.nodes:
        for i, c in enumerate(communities):
            if node in c:
                labels.append(i)

    # Pha 2
    nextG = nx.empty_graph(len(communities), nx.MultiGraph)
    for e in G.edges:
        for i in range(len(communities)):
            for j in range(len(communities)):
                if e[0] in communities[i] and e[1] in communities[j]:
                    nextG.add_edge(i, j)

    # Shuffle colors for better visualization
    palette = np.unique(labels)
    key = np.random.permutation(palette)
    labels = key[np.digitize(labels, palette, right=True)]

    return communities, labels, nextG

Bây giờ, bạn hoàn thành hàm `unfoldedCommunities` mà nhận đầu vào là một đồ thị, thực hiện Pha 1 và trả về mảng numpy array của các cộng đồng của đỉnh.

Ví dụ: nếu ta có hai cộng đồng, cộng đồng 1 - [1,2] và cộng đồng 2 - [3,4,5] thì đầu ra có dạng là [[1,2],[3,4,5]].

In [ ]:
def unfoldedCommunities(G):
    communities = [[n] for n in G.nodes]  # initial partition
    prev_max_modularity = -np.inf
    max_modularity = nx.algorithms.community.modularity(G, communities)
    while max_modularity > prev_max_modularity:
        prev_max_modularity = max_modularity
        for node in np.random.permutation(G.nodes):
            '''
            1) Remove the node from the initial community.
            2) Iterate all neighboring communities and move the node
               into a community with the maximum modularity gain. If
               there is no modularity gain, return the node into the
               initial community.
            '''
            # YOUR CODE HERE
            raise NotImplementedError()
    return [c for c in communities if len(c)]

In [ ]:
# Test
n = 4
fractalG = fractalGraph(n, 2, nx.complete_graph(n))
iterG = fractalG.copy()
communities, labels, nextG = louvainMethod(iterG)

assert len(set(labels)) == 16
assert len(set(labels[52:56])) == 1
assert len(set(labels[52:57])) == 2
assert nextG.number_of_nodes() == 16
assert round(np.linalg.det(nx.to_numpy_array(nextG)) * 1e-10, 4) == 136.5257

Chúng ta hãy xem các lần lặp lại đang diễn ra như thế nào. Đối với mỗi lần lặp, chúng tôi vẽ một đồ thị trong đó màu nút tương ứng với cộng đồng chưa được mở.

In [ ]:
n = 4
fractalG = fractalGraph(n, 3, nx.complete_graph(n))
iterG = fractalG

plt.figure(figsize=(7, 21))
for i in range(3):
    plt.subplot(3, 1, i+1)
    communities, labels, nextG = louvainMethod(iterG)
    iter_pos = nx.kamada_kawai_layout(iterG)
    nodes = nx.draw_networkx_nodes(
        iterG,
        iter_pos,
        cmap=plt.cm.rainbow,
        node_color=labels,
        edgecolors='black',
        node_size=100)
    nx.draw_networkx_edges(
        iterG,
        iter_pos,
        node_size=100)
    plt.axis('off')
    plt.title(
        '{} nodes, {} communities \nModularity {:.2f}'.format(
            len(iterG), len(communities), nx.community.modularity(iterG, communities)))
    iterG = nextG

plt.tight_layout()
plt.show()

Bạn có nhận xét gì?

YOUR ANSWER HERE

## Task 10. Phân hoạch "bản ngã" (Ego-Splitting)

Phân hoạch "bản ngã" hay Ego-Splitting là một framework cho việc phát hiện các cộng đồng trong mạng lưới phức. Nó tận dụng các cấu trúc cục bộ được biết là "ego-nets", tức là đồ thị con suy dẫn bởi láng giềng của nó, để phát hiện các cụm chồng chéo lên nhau. Ego-Splitting là một framework có tính mở rộng và linh hoạt cao, với những đảm bảo về mặt lý thuyết có thể chứng minh được, giúp giảm vấn đề phân cụm chồng chéo phức tạp thành vấn đề không chồng chéo (phân vùng) đơn giản hơn và dễ giải quyết hơn.

Xem xét một đồ thị như sau:

In [ ]:
adjlist = [
    'a b c e h g i',
    'f b c e h g i',
    'b e c',
    'g i h',
    'e c d',
    'c d',
    'i h j k',
    'h j',
    'j d k',
    'd k'
]

G = nx.parse_adjlist(adjlist, delimiter=' ')
pos = nx.kamada_kawai_layout(G)

plt.figure(figsize=(5, 5))
nx.draw_networkx(
    G,
    pos=pos,
    node_size=400,
    node_color='lightgray',
    edge_color='black',
    with_labels=True,
    font_size=10
)
plt.axis('off')
plt.show()

Thuật toán ego-splitting xử lý một đồ thị $G = (V, E)$ và cho ra kết quả là một tập hợp các cluster $S$ như sau:
- Bước 1: Với mỗi đỉnh $u$, ta xử dụng thuật toán gom cụm cục bộ để phân hoạch ego-net của $u$.
- Bước 2: Tạo ra một tập personas $V'$. Mỗi đỉnh $u$ trong $V$ sẽ tương ứng với personas $t_u$ trong $V'$.
- Bước 3: Thêm cạnh giữa các personas.
- Bước 4: Áp dụng thuật toán gom cụm toàn cục.

In [ ]:
def egoSplitting(graph):
    ego_nets = generateEgoNets(graph)
    persona_graph = generatePersonaGraph(ego_nets)
    return personaClustering(personaG)

Để hoàn thành, bạn cần hoàn thành một hàm `generateEgoNets` mà nhận đầu vào là một đồ thị và trả về một từ điển trong đó các khóa là các đỉnh và giá trị của chúng là ego-nets:

```
{'a': ego-net of the node a, 'b': ego-net of the node b, ...}
```

_Gợi ý: Bạn cần sử dụng `nx.ego_graph`_


In [ ]:
def generateEgoNets(graph: nx.Graph) -> Dict:
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test
ego_nets = generateEgoNets(G)
assert set(ego_nets.keys()) == {'a', 'b', 'c',
                                'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k'}
assert set(ego_nets['d'].nodes) == {'c', 'e', 'j', 'k'}
assert set(ego_nets['f'].nodes) == {'b', 'c', 'e', 'g', 'h', 'i'}

In [ ]:
nx.draw_networkx(ego_nets['f'], pos=nx.kamada_kawai_layout(G),
                 node_color='lightgray', node_size=400)
plt.title('Ego-net of the node f')
plt.axis('off')
plt.show()

Tiếp theo, chúng ta cần chia tách từng ego-net bằng bất kỳ phương pháp phân cụm nào. Giả sử đó là phương pháp phát hiện thành phần kết nối đơn giản.

In [ ]:
def egoClustering(graph):
    return [cc for cc in nx.connected_components(graph)]

Tiếp, bạn hoàn thành hàm `generatePersonaGraph` mà nhận đầu vào là một từ điển ego-net và trả về một đồ thị persona:
- Bước 1: Áp dụng gom cụm ego cho mỗi ego-net
- Bước 2: Thêm một bản sao của ego-node vào mỗi cluster và liên kết nó với từng thành viên. Nếu có hơn một cluster thì bản sao được đặt tên là node name + cluster index. Lấy ví dụ, nếu ego-net `a` cho các cụm `[b, c]` và `[d]` thì ta thêm cạnh `[a0, b]`, `[a0, c]`, và `[a1, d]`. Ngược lại, tên bản sao sẽ được đặt là tên của đỉnh.
- Bước 3: Hội tất cả các ego-nets vào đồ thị persona. Loại bỏ tất cả đỉnh mà có bản sao. Lấy vị, nếu đồ thị có các đỉnh `a`, `a0`, `a1`, thì ta sẽ loại bỏ `a`.

In [ ]:
def generatePersonaGraph(ego_nets: Dict) -> nx.Graph:
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test
ego_nets = generateEgoNets(G)
personaG = generatePersonaGraph(ego_nets)
assert set(personaG.nodes) == {'a0', 'a1', 'b', 'c',
                               'd0', 'd1', 'e', 'f0', 'f1', 'g', 'h', 'i', 'j', 'k'}
assert {i[0] for i in personaG.neighbors('k')} == {'d', 'i', 'j'}
assert {i[0] for i in personaG.neighbors('g')} == {'a', 'f', 'h', 'i'}

Cuối cùng, ta cần áp dụng bất kỳ thuật toán gom cụm nào cho đồ thị persona. Khuyến khích xài những thuật toán đơn giản :). Và ta cũng cần loại bỏ các chỉ số cụm từ tên đỉnh nữa.

In [ ]:
def personaClustering(graph: nx.Graph) -> List:
    communities = [cc for cc in nx.connected_components(graph)]
    for c in communities:
        for node in c:
            c.remove(node)
            c.add(node[0])
    return communities

In [ ]:
plt.figure(figsize=(15, 5))
pos = nx.kamada_kawai_layout(G)
c_colors = ['tab:orange', 'tab:blue', 'tab:green']
for i, c in enumerate(egoSplitting(G)):
    plt.subplot(1, 3, i+1)
    color = []
    for node in G.nodes:
        color.append(c_colors[i] if node in c else 'white')
    nx.draw_networkx(G, pos=pos, node_color=color,
                     edgecolors='black', node_size=400)
    plt.axis('off')
    plt.title('Colored nodes are in the community {}'.format(i+1))

## Task 11. Lan truyền nhãn (Label propogation)


Phương pháp lan truyền nhãn dựa trên ý tương đơn giản như sau: mỗi đỉnh được khởi tạo với một nhãn duy nhất và tại mỗi bước thì mỗi nút sẽ nhận được nhãn mà hầu hết các láng giềng của nó đang có.

Để hoàn thành phần này, bạn cần code hàm `updateLabels` mà nhận đầu vào là một đồ thị, mảng numpy array theo thứ tự `graph.nodes` và trả về các nhãn mới. Nhãn của đỉnh là một nhãn ngẫu nhiên xuất hiện với tần số cao giữa các láng giềng của nó ở bước trước đó. Thứ tự trong `n` đỉnh của mạng được cập nhật tại mỗi bước lặp thì được chọn ngẫu nhiên.

In [ ]:
def updateLabels(graph, labels):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Test
G = nx.karate_club_graph()
labels = np.array(list(G.nodes)) # initial partition
new_labels = updateLabels(G, labels)
assert new_labels.shape == (34, )
assert len(set(new_labels)) < len(set(labels))
G = nx.star_graph(4)
labels = np.array(list(G.nodes)) # initial partition
for _ in range(5):
    labels = updateLabels(G, labels)
new_labels = updateLabels(G, labels)
assert len(set(new_labels - labels)) == 2
assert np.sum(list(set(new_labels - labels))) == 0

In [ ]:
def plotPropogation(G, update_method, n=50):
    pos=nx.kamada_kawai_layout(G)
    labels = np.array(list(G.nodes))
    for i in range(n):
        clear_output(wait=True)
        labels = update_method(G, labels)
        plt.figure(figsize=(6, 6))
        nx.draw_networkx_nodes(G, pos=pos, node_color=labels, cmap=plt.cm.tab10_r)
        nx.draw_networkx_edges(G, pos=pos, alpha=0.2)
        nx.draw_networkx_labels(G, pos=pos)
        plt.axis('off')
        plt.title('Iteration {}/{}'.format(i+1, n))
        plt.show()

In [ ]:
plotPropogation(nx.karate_club_graph(), updateLabels)

Có một vấn đề ở đây là các đồ thị con trong mạng mà là lưỡng phân hoặc xém lưỡng phân trong cấu trúc lẫn đến hiện tượng dao động nhãn (oscillations labels). Điều này đúng trong một số trường hợp mà các cộng đồng được lấy từ một đồ thị hình sao.

In [ ]:
plotPropogation(nx.star_graph(30), updateLabels, 10)

Vậy thì phải làm sao đây?

Do đó, ta cần một cơ chế cập nhật bất đồng bộ (asynchronous updating) mà trong đó một số láng giềng đã có được cập nhật lại bước lặp hiện tại trong khi một số khác thì không.

Để làm được điều này, bạn cần viết một hàm `asyncUpdateLabels` mà tạo ra việc cập nhất bất đồng bộ. Thứ tự trong tất cả `n` đỉnh của mạng được cập nhật tại mỗi lần lặp được chọn một cách ngẫu nhiên.

In [ ]:
def asyncUpdateLabels(graph, labels):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
G = nx.karate_club_graph()
labels = np.array(list(G.nodes))  # initial partition
new_labels = asyncUpdateLabels(G, labels)
assert new_labels.shape == (34, )
assert len(set(new_labels)) < len(set(labels))
G = nx.star_graph(4)
labels = np.array(list(G.nodes))  # initial partition
for _ in range(5):
    labels = asyncUpdateLabels(G, labels)
new_labels = asyncUpdateLabels(G, labels)
assert np.all(new_labels - labels == 0)

In [ ]:
plotPropogation(nx.star_graph(30), asyncUpdateLabels, 10)

In [ ]:
plotPropogation(nx.karate_club_graph(), asyncUpdateLabels, 30)